In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Telomere_Length/GSE16058'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Distinctions between the stasis and telomere attrition senescence barriers in cultured human mammary epithelial cells"
!Series_summary	"Molecular distinctions between the stasis and telomere attrition senescence barriers in cultured human mammary epithelial cells"
!Series_summary	""
!Series_summary	"Normal human epithelial cells in culture have generally shown a limited proliferative potential of ~10-40 population doublings before encountering a stress-associated senescence barrier (stasis) associated with elevated levels of cyclin-dependent kinase inhibitors p16 and/or p21. We now show that simple changes in media composition can expand the proliferative potential of human mammary epithelial cells (HMEC) initiated as primary cultures to 50-60 population doublings, followed by p16(+), senescence-associated b-galactosidase(+) stasis. We compared the properties of growing and senescent pre-stasis HMEC with growing and senescent post-selection HMEC, 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is present in the dataset
if 'gene expression' in "!Series_title".lower() or 'gene expression' in "!Series_summary".lower():
    is_gene_available = True

# Based on sample characteristics dictionary observation
sample_characteristics = {
    0: ['cell type: mammary epithelial cell', 'cell type: mammary fibroblast cell'],
    1: ['individual: 184', 'individual: 48', 'individual: 240L', 'individual: 250MK'],
    2: ['passage: 2p', 'passage: 4p', 'passage: 6p', 'passage: 9p', 'passage: 11p', 'passage: 14p', 'passage: 8p', 'passage: 22p', 'passage: 12p', 'passage: 3p', 'passage: 5p', 'passage: 10p', 'passage: 15p', 'passage: 16p', 'passage: 7p', 'passage: 21p'],
    3: ['growth status: Growing-Prestasis', 'growth status: Intermediate-Prestasis', 'growth status: Stasis', 'growth status: Prestasis', 'growth status: PostSelection', 'growth status: Agonesence-Postselection', 'growth status: Growing-Postselection', 'growth status: Growing', 'growth status: Senescent']
}

# As per the available data, there are no explicit keys indicating 'Telomere_Length', 'age', or 'gender'
trait_row = age_row = gender_row = None

# Define conversion functions if required data is available
def convert_trait(value):
    try:
        return float(value.split(':')[1].strip())
    except (ValueError, IndexError):
        return None

def convert_age(value):
    try:
        return float(value.split(':')[1].strip())
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    gender = value.split(':')[1].strip().lower()
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    else:
        return None

# Save cohort info as per instruction
save_cohort_info('GSE16058', './preprocessed/Telomere_Length/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction step, conditional execution only if trait_row is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Telomere_Length', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Telomere_Length/trait_data/GSE16058.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
